# Testing

A notebook to test the implementation

In [1]:
import gzip
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
model_test = tf.keras.Sequential([tf.keras.layers.Flatten(input_shape=(28, 28)),
                                 tf.keras.layers.Dense(300, activation=tf.nn.relu),
                                 tf.keras.layers.BatchNormalization()])
model_test.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 batch_normalization (BatchN  (None, 300)              1200      
 ormalization)                                                   
                                                                 
Total params: 236,700
Trainable params: 236,100
Non-trainable params: 600
_________________________________________________________________


## 0 Preprocessing

For testing, we use the `MNIST` data. We first extract the data from `gzip` file.

In [3]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return (tf.cast(image, tf.float32)/255.0 , label)  

def preProcess(dset, batch_size):
  '''Pre-processes the dataset.'''
  dset = dset.map(normalize_img)
  dset = dset.shuffle(len(dset))
  dset = dset.batch(batch_size)
  return dset
  
# prepare data
def get_imgs(dir):
    with gzip.open(dir, 'r') as f:
        # first 4 bytes (some number)
        _ = int.from_bytes(f.read(4), 'big')
        # no. of images
        num_imgs = int.from_bytes(f.read(4), 'big')
        # row count
        row_cnt = int.from_bytes(f.read(4), 'big')
        # column count
        col_cnt = int.from_bytes(f.read(4), 'big')

        img_data = f.read()
        images = np.frombuffer(img_data, dtype=np.uint8).\
            reshape((num_imgs, row_cnt, col_cnt))
        return images

def get_labels(dir):
    with gzip.open(dir, 'r') as f:
        _ = int.from_bytes(f.read(4), 'big')
        label_cnt = int.from_bytes(f.read(4), 'big')
        print(label_cnt)
        label_data = f.read()
        labels = np.frombuffer(label_data, dtype=np.uint8)
        return labels

We now make `dataset`. Note that we used the `tf.data.Dataset.from_tensor_slices` to create dataset from a tuple of numpy arrays; if they were instead lists, then there will be an error

In [4]:
def getLocalDset(batch_size = 128):
    imgs     = get_imgs('data/train-images-idx3-ubyte.gz')
    labels   = get_labels('data/train-labels-idx1-ubyte.gz')
    split    = int(0.75*len(imgs))
    ds_train = tf.data.Dataset.from_tensor_slices((imgs[:split], labels[:split]))
    ds_test  = tf.data.Dataset.from_tensor_slices((imgs[split:], labels[split:]))
    print(len(ds_train))
    print(len(ds_test))
    return preProcess(ds_train, batch_size), preProcess(ds_test, batch_size)

We can also get the data by downloading

In [5]:
def fetch_img(ds, batch_size = 128):
  ds = ds.cache()
  # ds = ds.shuffle(ds_info.splits['train'].num_examples)
  ds = ds.batch(batch_size)
  # ds = ds.prefetch(tf.data.AUTOTUNE)
  return ds

def getRemoteDset(batch_size = 128, name = 'MNIST'):

  '''Prepares the MNIST dataset for training.'''  

  if name == 'MNIST':
    (ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    # split=['test', 'train[0%:10%]','train[10%:]'],
    split = ['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info = True
    )

  ds_train = ds_train.map(normalize_img, 
                          num_parallel_calls=tf.data.AUTOTUNE)
  ds_test = ds_test.map(normalize_img, 
                          num_parallel_calls=tf.data.AUTOTUNE)
  ds_train = fetch_img(ds_train, batch_size)
  ds_test = fetch_img(ds_test, batch_size)

  return (ds_train, ds_test), ds_info

(ds_train, ds_test), ds_info = getRemoteDset()

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\mnist\3.0.1.incompleteAFUHB9\mnist-train.tfrecord*...:   0%|          | 0/6000…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\mnist\3.0.1.incompleteAFUHB9\mnist-test.tfrecord*...:   0%|          | 0/10000…

Dataset mnist downloaded and prepared to ~\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


## 1 Running experiment

First define some hyperparameters

In [6]:
layers = [300, 100, 10] # use [784, 100, 10]

activation = 'relu'

lr = 0.001

patience = 3

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

metrics = tf.keras.metrics.SparseCategoricalAccuracy()    

train_loss = tf.keras.metrics.Mean

train_acc = tf.keras.metrics.SparseCategoricalAccuracy  

epochs = 1 # use a much bigger int

num_pruning = 1 # use 5 or more  

step_perc = 0.5

In [7]:
from pruning import *
from models import *

model_params = {'layers': layers,
                'activation': 'relu',
                'BatchNorm':True,
                'Dropout': [0.5,0.5,0.4],
                'optimizer': tf.keras.optimizers.Adam(lr),
                'loss_fn': loss_fn,
                'metrics': metrics
                }

train_params = {'train_loss': train_loss,
                'train_acc': train_acc,
                'patience': 3
                }

prune_exp = pruning(ds_train, ds_test, model_params,
                    train_params,
                    epochs, num_pruning, step_perc)
                    
# prune_exp.test_training()
# prune_exp.test_model()

In [8]:
prune_exp.prune()


 
 Iterative pruning round: 0 
 


 
 After pruning, the total no. of nonzero weights is: 266200 
 


 Start original model training. 

Epoch 0 for original
OG train acc 0.5376333594322205
Original model accuracy 0.9380000233650208 


 
 After pruning, the total no. of nonzero weights is: 133100 
 


 Start Lottery ticket training 

Epoch 0 for lottery ticket
Ticket train acc 0.531416654586792 

Lottery ticket accuracy 0.9322999715805054 


 Lottery ticket training finished. Highest Accuracy 0.9322999715805054 

INFO:tensorflow:Assets written to: saved_models/ticket_acc_0.9322999715805054 pruning round_0\assets


INFO:tensorflow:Assets written to: saved_models/ticket_acc_0.9322999715805054 pruning round_0\assets


In [9]:
prune_exp.removeLogs()

Removing 20220823-084832


In [10]:
prune_exp.removeModels()

Removing ticket_acc_0.9322999715805054 pruning round_0
